In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F
from torch.utils.data import Subset


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image

import os
from torchsummary import summary



# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/


# Corrupted CIFAR10

In [2]:
CIFAR10C_corruptions = ["brightness", "contrast", "defocus_blur", "elastic_transform", 
               "fog", "frost", "gaussian_blur", "gaussian_noise","glass_blur",
               "impulse_noise", "jpeg_compression", "motion_blur",
               "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise","zoom_blur"]

#https://github.com/tanimutomo/cifar10-c-eval/blob/master
class CIFAR10C(datasets.VisionDataset):
    def __init__(self, root :str, name :str,transform):

        corruptions = ["brightness", "contrast", "defocus_blur", "elastic_transform", 
               "fog", "frost", "gaussian_blur", "gaussian_noise","glass_blur",
               "impulse_noise", "jpeg_compression", "motion_blur",
               "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise","zoom_blur"]
        
        assert name in corruptions
        super(CIFAR10C, self).__init__(root, transform=transform)
        data_path = os.path.join(root, name + '.npy')
        target_path = os.path.join(root, 'labels.npy')
        
        self.data = np.load(data_path)
        self.targets = np.load(target_path)
        
    def __getitem__(self, index):
        img, targets = self.data[index], self.targets[index]
        img = Image.fromarray(img)
        
        img = self.transform(img)

        return img, targets
    
    def __len__(self):
        return len(self.data)

def CIFAR10C_DataLoader(root, batch_size=64):
    transformList = [transforms.ToTensor(),transforms.Normalize((0.491, 0.482, 0.446), (0.247, 0.243, 0.261))]
    transform = transforms.Compose(transformList)
    Dataloaders={}

    CIFAR10C_corruptions = ["brightness", "contrast", "defocus_blur", "elastic_transform", 
               "fog", "frost", "gaussian_blur", "gaussian_noise","glass_blur",
               "impulse_noise", "jpeg_compression", "motion_blur",
               "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise","zoom_blur"]
    
    for cname in CIFAR10C_corruptions:
        dataset = CIFAR10C(root,cname,transform=transform)
        CIFARC_Loader=torch.utils.data.DataLoader(dataset, batch_size=256,shuffle=False, num_workers=2)
        Dataloaders[cname]=CIFARC_Loader

    return Dataloaders

root="./Datasets/CIFAR-10/CIFAR-10-C/"
Dataloaders=CIFAR10C_DataLoader(root)

In [3]:
for index, (data, target) in enumerate(Dataloaders["motion_blur"]):
    break

# CORRUPTED TINY IMAGENET

In [8]:
def TinyImagenetC_DataLoader(root="./Datasets/TINY-IMAGENET/TinyImageNet-C/Tiny-ImageNet-C",batch_size=64):

    TinyImagenetC_corruptions = ["brightness", "contrast", "defocus_blur", "elastic_transform", 
               "fog", "frost", "gaussian_blur", "gaussian_noise","glass_blur",
               "impulse_noise", "jpeg_compression", "motion_blur",
               "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise","zoom_blur"]

    SeverityLevels=["1","2","3","4","5"]

    transformList = [transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
    transform = transforms.Compose(transformList)

    Dataloaders={}
    for cname in TinyImagenetC_corruptions:
        Level_Dataloaders={}
        for clevel in SeverityLevels:
            data_path=os.path.join(root, cname,clevel)
            Dataset = datasets.ImageFolder(data_path, transform=transform)
            Dataloader = torch.utils.data.DataLoader(Dataset, batch_size=batch_size,shuffle=True, num_workers=2)
            Level_Dataloaders[clevel]=Dataloader #Levels for individual corruptions

        Dataloaders[cname]=Level_Dataloaders #All Corruptions and their levels

    return Dataloaders

Dataloaders=TinyImagenetC_DataLoader()

In [27]:
for index, (data, target) in enumerate(Dataloaders["frost"]["2"]):
    break

In [28]:
target

tensor([149, 293, 133, 637, 950, 242, 263, 101,  73,  98, 874, 167, 770, 144,
        608, 875, 823, 246, 891, 146, 813, 545, 554, 653, 401, 182, 312, 478,
        319, 926, 423, 420, 637, 665, 204, 184,  88, 950, 549, 452, 736, 614,
        568, 658, 323, 654,  76, 820, 713, 870, 480, 555, 994, 271, 315, 684,
        938, 412, 194, 434, 215, 821, 587, 580])